<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/prepare_data_momentum_algo_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_ta --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from statsmodels.regression.rolling import RollingOLS
#import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

In [8]:
def get_nifty50_tickers():
    #data = pd.read_csv('https://raw.githubusercontent.com/anirbanghoshsbi/data/main/STOCKS100.txt', sep='\t')
    nifty50_url = "https://raw.githubusercontent.com/anirbanghoshsbi/data/main/ind_nifty50list.csv"
    nifty50 = pd.read_csv(nifty50_url)
    tickers = nifty50['Symbol'].apply(lambda x: x + ".NS").tolist()
    #reject_lst=['MOTHERSUMI.NS', 'CADILAHC.NS', 'ADANITRANS.NS', 'SRTRANSFIN.NS', 'INFRATEL-EQ.NS','HDFC.NS','LTI.NS']
    #nifty_final= [item for item in nifty50 if item not in reject_lst]

    return tickers


In [9]:
symbols_list = get_nifty50_tickers()

end_date = '2023-09-27'

start_date = '2021-01-02'
df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()



[*********************100%%**********************]  50 of 50 completed


In [11]:
df

Price                       adj close        close         high          low  \
date       ticker                                                              
2021-01-04 ADANIENT.NS     493.269440   494.500000   502.600006   486.649994   
           ADANIPORTS.NS   486.647064   498.899994   510.899994   495.200012   
           APOLLOHOSP.NS  2387.047852  2409.250000  2429.000000  2398.250000   
           ASIANPAINT.NS  2668.799072  2753.699951  2791.100098  2745.000000   
           AXISBANK.NS     621.989929   624.700012   633.000000   621.299988   
...                               ...          ...          ...          ...   
2023-09-26 TECHM.NS       1245.547852  1282.300049  1303.050049  1279.150024   
           TITAN.NS       3251.959229  3262.600098  3308.000000  3255.000000   
           ULTRACEMCO.NS  8228.900391  8228.900391  8328.000000  8206.200195   
           UPL.NS          617.549988   617.549988   623.000000   615.000000   
           WIPRO.NS        412.021301   412.899994   416.799988   411.700012   

Price                            open    volume  
date       ticker                                
2021-01-04 ADANIENT.NS     492.000000   4936582  
           ADANIPORTS.NS   507.000000   9998803  
           APOLLOHOSP.NS  2418.100098    527018  
           ASIANPAINT.NS  2781.100098   1360486  
           AXISBANK.NS     627.799988  14068156  
...                               ...       ...  
2023-09-26 TECHM.NS       1300.000000   1141475  
           TITAN.NS       3294.000000   1054827  
           ULTRACEMCO.NS  8237.000000    515796  
           UPL.NS          616.950012   1906824  
           WIPRO.NS        416.799988   3240283  

[33850 rows x 6 columns]

In [12]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])

df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])

df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price                       adj close        close         high          low  \
date       ticker                                                              
2021-01-04 ADANIENT.NS     493.269440   494.500000   502.600006   486.649994   
           ADANIPORTS.NS   486.647064   498.899994   510.899994   495.200012   
           APOLLOHOSP.NS  2387.047852  2409.250000  2429.000000  2398.250000   
           ASIANPAINT.NS  2668.799072  2753.699951  2791.100098  2745.000000   
           AXISBANK.NS     621.989929   624.700012   633.000000   621.299988   
...                               ...          ...          ...          ...   
2023-09-26 TECHM.NS       1245.547852  1282.300049  1303.050049  1279.150024   
           TITAN.NS       3251.959229  3262.600098  3308.000000  3255.000000   
           ULTRACEMCO.NS  8228.900391  8228.900391  8328.000000  8206.200195   
           UPL.NS          617.549988   617.549988   623.000000   615.000000   
           WIPRO.NS        412.021301   412.899994   416.799988   411.700012   

Price                            open    volume  garman_klass_vol        rsi  \
date       ticker                                                              
2021-01-04 ADANIENT.NS     492.000000   4936582          0.000517        NaN   
           ADANIPORTS.NS   507.000000   9998803         -0.000161        NaN   
           APOLLOHOSP.NS  2418.100098    527018          0.000017        NaN   
           ASIANPAINT.NS  2781.100098   1360486         -0.000518        NaN   
           AXISBANK.NS     627.799988  14068156          0.000141        NaN   
...                               ...       ...               ...        ...   
2023-09-26 TECHM.NS       1300.000000   1141475         -0.000536  60.191945   
           TITAN.NS       3294.000000   1054827          0.000067  63.085739   
           ULTRACEMCO.NS  8237.000000    515796          0.000108  46.307193   
           UPL.NS          616.950012   1906824          0.000083  51.409031   
           WIPRO.NS        416.799988   3240283          0.000024  45.229151   

Price                       bb_low    bb_mid   bb_high       atr      macd  \
date       ticker                                                            
2021-01-04 ADANIENT.NS         NaN       NaN       NaN       NaN       NaN   
           ADANIPORTS.NS       NaN       NaN       NaN       NaN       NaN   
           APOLLOHOSP.NS       NaN       NaN       NaN       NaN       NaN   
           ASIANPAINT.NS       NaN       NaN       NaN       NaN       NaN   
           AXISBANK.NS         NaN       NaN       NaN       NaN       NaN   
...                            ...       ...       ...       ...       ...   
2023-09-26 TECHM.NS       7.060202  7.111689  7.163176 -0.836801  0.791820   
           TITAN.NS       8.019385  8.069376  8.119368 -0.306329  1.084957   
           ULTRACEMCO.NS  8.999038  9.039667  9.080296 -0.866602 -0.191764   
           UPL.NS         6.382715  6.419798  6.456881 -1.617716  0.215990   
           WIPRO.NS       6.003548  6.054552  6.105556 -0.867955  0.173451   

Price                     dollar_volume  
date       ticker                        
2021-01-04 ADANIENT.NS      2435.065037  
           ADANIPORTS.NS    4865.888126  
           APOLLOHOSP.NS    1258.017185  
           ASIANPAINT.NS    3630.863775  
           AXISBANK.NS      8750.251354  
...                                 ...  
2023-09-26 TECHM.NS         1421.761734  
           TITAN.NS         3430.254397  
           ULTRACEMCO.NS    4244.433906  
           UPL.NS           1177.559138  
           WIPRO.NS         1335.065618  

[33850 rows x 14 columns]

In [13]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('W').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('W').last().stack('ticker')],
                  axis=1)).dropna()

data

dollar_volume    adj close       atr   bb_high  \
date       ticker                                                          
2021-02-14 ADANIENT.NS      5128.580459   717.160950 -1.185551  6.536214   
           ADANIPORTS.NS    8723.193572   601.456360 -0.271290  6.390277   
           APOLLOHOSP.NS    1989.958667  2722.775635 -1.109806  7.933740   
           ASIANPAINT.NS    8748.006235  2409.449219  0.710650  7.872057   
           AXISBANK.NS     11904.589680   747.144653  2.212232  6.651007   
...                                 ...          ...       ...       ...   
2023-10-01 TECHM.NS         2036.387946  1245.547852 -0.836801  7.163176   
           TITAN.NS         2735.584163  3251.959229 -0.306329  8.119368   
           ULTRACEMCO.NS    4040.067485  8228.900391 -0.866602  9.080296   
           UPL.NS           1299.316888   617.549988 -1.617716  6.456881   
           WIPRO.NS         1628.383344   412.021301 -0.867955  6.105556   

                            bb_low    bb_mid  garman_klass_vol      macd  \
date       ticker                                                          
2021-02-14 ADANIENT.NS    6.136371  6.336293          0.002082  0.241983   
           ADANIPORTS.NS  6.185985  6.288131          0.001643  1.103108   
           APOLLOHOSP.NS  7.831898  7.882819          0.000164  0.521738   
           ASIANPAINT.NS  7.720908  7.796483         -0.000472 -1.816223   
           AXISBANK.NS    6.434630  6.542819          0.000271  1.869786   
...                            ...       ...               ...       ...   
2023-10-01 TECHM.NS       7.060202  7.111689         -0.000536  0.791820   
           TITAN.NS       8.019385  8.069376          0.000067  1.084957   
           ULTRACEMCO.NS  8.999038  9.039667          0.000108 -0.191764   
           UPL.NS         6.382715  6.419798          0.000083  0.215990   
           WIPRO.NS       6.003548  6.054552          0.000024  0.173451   

                                rsi  
date       ticker                    
2021-02-14 ADANIENT.NS    86.404774  
           ADANIPORTS.NS  74.374528  
           APOLLOHOSP.NS  63.928984  
           ASIANPAINT.NS  40.716763  
           AXISBANK.NS    65.886310  
...                             ...  
2023-10-01 TECHM.NS       60.191945  
           TITAN.NS       63.085739  
           ULTRACEMCO.NS  46.307193  
           UPL.NS         51.409031  
           WIPRO.NS       45.229151  

[6900 rows x 9 columns]

In [14]:
def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:

        df[f'return_{lag}m'] = (df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df


data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

dollar_volume    adj close       atr   bb_high  \
date       ticker                                                          
2021-05-09 ADANIENT.NS     10175.488029  1295.667725 -0.561789  7.173569   
           ADANIPORTS.NS   17638.086884   748.894104  1.314011  6.668692   
           APOLLOHOSP.NS    3192.961695  3280.339355 -0.582096  8.118916   
           ASIANPAINT.NS    3106.315739  2472.978271 -0.121263  7.854967   
           AXISBANK.NS      7558.309322   713.640625  0.825621  6.603343   
...                                 ...          ...       ...       ...   
2023-10-01 TECHM.NS         2036.387946  1245.547852 -0.836801  7.163176   
           TITAN.NS         2735.584163  3251.959229 -0.306329  8.119368   
           ULTRACEMCO.NS    4040.067485  8228.900391 -0.866602  9.080296   
           UPL.NS           1299.316888   617.549988 -1.617716  6.456881   
           WIPRO.NS         1628.383344   412.021301 -0.867955  6.105556   

                            bb_low    bb_mid  garman_klass_vol      macd  \
date       ticker                                                          
2021-05-09 ADANIENT.NS    6.974810  7.074190          0.000279  0.417129   
           ADANIPORTS.NS  6.524793  6.596742          0.000924  0.190721   
           APOLLOHOSP.NS  7.988927  8.053921          0.000245  0.566654   
           ASIANPAINT.NS  7.793853  7.824410         -0.000233  0.109350   
           AXISBANK.NS    6.449665  6.526504          0.000120  0.080523   
...                            ...       ...               ...       ...   
2023-10-01 TECHM.NS       7.060202  7.111689         -0.000536  0.791820   
           TITAN.NS       8.019385  8.069376          0.000067  1.084957   
           ULTRACEMCO.NS  8.999038  9.039667          0.000108 -0.191764   
           UPL.NS         6.382715  6.419798          0.000083  0.215990   
           WIPRO.NS       6.003548  6.054552          0.000024  0.173451   

                                rsi  return_1m  return_2m  return_3m  \
date       ticker                                                      
2021-05-09 ADANIENT.NS    71.319236   0.120998   0.059275   0.038717   
           ADANIPORTS.NS  56.033368   0.051640   0.028812   0.005376   
           APOLLOHOSP.NS  62.275292   0.035449   0.017165   0.026499   
           ASIANPAINT.NS  50.734795   0.006013   0.006670  -0.014223   
           AXISBANK.NS    54.216661   0.002588   0.033259   0.023145   
...                             ...        ...        ...        ...   
2023-10-01 TECHM.NS       60.191945  -0.017658  -0.006850   0.005081   
           TITAN.NS       63.085739  -0.003634   0.002152   0.006573   
           ULTRACEMCO.NS  46.307193   0.005069  -0.029032  -0.008037   
           UPL.NS         51.409031   0.004473  -0.013564   0.005871   
           WIPRO.NS       45.229151  -0.013381  -0.032439  -0.013436   

                          return_6m  return_9m  return_12m  
date       ticker                                           
2021-05-09 ADANIENT.NS     0.046601   0.040341    0.050525  
           ADANIPORTS.NS   0.014781   0.002758    0.018438  
           APOLLOHOSP.NS   0.025194   0.015631    0.015646  
           ASIANPAINT.NS   0.003070   0.007434    0.002171  
           AXISBANK.NS     0.004380  -0.002147   -0.003816  
...                             ...        ...         ...  
2023-10-01 TECHM.NS        0.009608   0.018108    0.010855  
           TITAN.NS        0.011269   0.009854    0.003361  
           ULTRACEMCO.NS   0.000285  -0.001046   -0.000702  
           UPL.NS          0.010961   0.000450   -0.004551  
           WIPRO.NS        0.000485   0.003882    0.003520  

[6300 rows x 15 columns]

In [15]:
data.to_csv('nifty_50_weekly_data.csv')